In [1]:
import numpy as np
import pandas as pd
import os
import sys
import re
import datetime as dt
from time import mktime

#plotting
from matplotlib import pyplot as plt
import matplotlib.patches as mlines
import matplotlib.patches as mpatches
from matplotlib import colors as mcolors
import matplotlib.backends.backend_pdf
import matplotlib.font_manager as font_manager
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import AutoMinorLocator
from matplotlib import gridspec
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

# Useful Functions and Settings

In [2]:
# Set the font dictionaries (for plot title and axis titles)
title_font = {'fontname':'System San Francisco Display Regular', 'size':'16', 'color':'black', 'weight':'normal',
              'verticalalignment':'bottom'} # Bottom vertical alignment for more space
axis_font = {'fontname':'System San Francisco Display Bold', 'size':'14'}
font_path = "/global/homes/t/tkurth/fonts/System San Francisco Display Regular.ttf"
font_path_bf = "/global/homes/t/tkurth/fonts/System San Francisco Display Bold.ttf"
font_prop_title = font_manager.FontProperties(fname=font_path_bf, size=40)
font_prop_axis = font_manager.FontProperties(fname=font_path, size=30)
font_prop_axis_labels = font_manager.FontProperties(fname=font_path, size=40)
font_prop_legend = font_manager.FontProperties(fname=font_path, size=26)
#plt.style.use('seaborn-talk')
#plt.style.available

#some plot objects
lw=4
ms=250
colorlist=['crimson','dodgerblue','blueviolet','forestgreen','goldenrod','silver']+[str(name) for name,hex in mcolors.cnames.items()]
markerlist=['v','s','o','*','^','x','p','8','+','>','<']
width=0.4

## Process Output

In [3]:
def parse_tiramisu_file(filename):
    
    #parse file content
    with open(filename,'r') as f:
        body = f.readlines()
    
    #get number of ranks:
    ranklist = []
    num_ranks = 1
    for line in body:
        if "Using distributed computation with Horovod:" in line:
            search = re.findall(r'^Using distributed computation with Horovod: (.*?) total ranks.*',line)
            num_ranks = np.int(search[0])
            break
    
    #grab all lines with REPORT predicate
    reports = []
    for line in body:
        if "REPORT: rank" in line:
            #dissect that line
            search = re.findall(r'^REPORT: rank (.*?), training loss for step (.*?) \(of .*?\) is (.*?)\n',line)
            
            if search:
                rank = np.int(search[0][0])
                gstep = np.int(search[0][1])
                loss = np.float(search[0][2])
            
                reports.append({"rank": rank, 
                                "global_step": gstep,
                                "metric_type": "loss",
                                "metric_value": loss, 
                                "num_workers": num_ranks, 
                                "type": "training"})
        
        if "COMPLETED: rank" in line:
            #dissect that line
            search = re.findall(r'^COMPLETED: rank (.*?), evaluation loss for epoch (.*?) \(of .*?\) is (.*?)\n',line)
            
            if search:
                rank = np.int(search[0][0])
                gstep = np.int(search[0][1])
                loss = np.float(search[0][2])
            
                reports.append({"rank": rank, 
                                "global_step": gstep,
                                "metric_type": "loss",
                                "metric_value": loss, 
                                "num_workers": num_ranks, 
                                "type": "evaluation"})
                
            search = re.findall(r'^COMPLETED: rank (.*?), evaluation IoU for epoch (.*?) \(of .*?\) is (.*?)\n',line)
            
            if search:
                rank = np.int(search[0][0])
                gstep = np.int(search[0][1])
                iou = np.float(search[0][2])
            
                reports.append({"rank": rank, 
                                "global_step": gstep,
                                "metric_type": "iou",
                                "metric_value": iou, 
                                "num_workers": num_ranks, 
                                "type": "evaluation"})
            
    return pd.DataFrame(reports)

In [14]:
def preprocess_tiramisu_frame(dflist, glist):
    #clean up
    datadf = pd.concat(dflist)
    datadf.sort_values(by=glist, inplace=True)
    datadf.reset_index(inplace=True, drop=True)
    
    return datadf

# Load Data

In [15]:
glist=["num_workers","type","metric_type","global_step","rank"]

In [16]:
dflist = []

datapath = './run_3'

files = [datapath+'/'+x for x in os.listdir(datapath) if re.match(r'slurm-\d{7,}\.out',x)]

for file in files:
    tmpdf = parse_tiramisu_file(file)
    dflist.append(tmpdf)

datadf = preprocess_tiramisu_frame(dflist, glist)

In [17]:
datadf

,global_step,metric_type,metric_value,num_workers,rank,type
0,0,iou,0.306559,1024,0,evaluation
1,0,iou,0.383319,1024,1,evaluation
2,0,iou,0.322549,1024,2,evaluation
3,0,iou,0.402201,1024,3,evaluation
4,0,iou,0.488878,1024,4,evaluation
5,0,iou,0.275482,1024,5,evaluation
6,0,iou,0.268025,1024,6,evaluation
7,0,iou,0.244259,1024,7,evaluation
8,0,iou,0.374785,1024,8,evaluation
9,0,iou,0.262073,1024,9,evaluation
